①データの読み込み・外観

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import seaborn as sns
matplotlib.style.use('ggplot')

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import datetime
%matplotlib inline

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

In [2]:
df = pd.read_csv(r"C:\Users\yuka0\Documents\skillupai\ML\DAY1\homework\ks-projects-201801.csv")

(6)募集期間

In [3]:
df['deadline'] = pd.to_datetime(df["deadline"])
df["launched"] = pd.to_datetime(df["launched"])

In [4]:
df["days"] = (df["deadline"] - df["launched"]).dt.days

In [5]:
df["days"].head()

0    58
1    59
2    44
3    29
4    55
Name: days, dtype: int64

③不要と思われるデータの削除

In [6]:
#stateが成功と失敗以外のデータを削除
#ID削除
#nameとcategoryも削除
#daysを募集期間で出したのでdf["deadline"]とdf["launched"]も削除
#backers,pledgedと、usd pledgedより右はモデルにいれちゃだめだと思うので削除

In [7]:
df = df[(df["state"] == "successful") | (df["state"] == "failed")]

In [8]:
df = df.drop(["ID","name","deadline","launched","backers","pledged","usd pledged","usd_pledged_real","usd_goal_real"], axis=1)

In [9]:
df["state"] = df["state"].replace("failed",0)
df["state"] = df["state"].replace("successful",1)

In [10]:
df.head()

,category,main_category,currency,goal,state,country,days
0,Poetry,Publishing,GBP,1000.0,0,GB,58
1,Narrative Film,Film & Video,USD,30000.0,0,US,59
2,Narrative Film,Film & Video,USD,45000.0,0,US,44
3,Music,Music,USD,5000.0,0,US,29
5,Restaurants,Food,USD,50000.0,1,US,34


④カテゴリ変数処理

In [11]:
df = pd.get_dummies(df,drop_first = True)

④標準化

In [12]:
stdsc = StandardScaler()
df["goal"] = stdsc.fit_transform(df[["goal"]].values)
df["days"] = stdsc.fit_transform(df[["days"]].values)

C:\Users\yuka0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\yuka0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [13]:
train_data = df.drop("state", axis=1)
y = df["state"].values
X = train_data.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

⑤ロジスティック回帰→L2正則化

ハイパーパラメーター

In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV

parameters = {'penalty':['l1', 'l2'], 'max_iter':[5000,30000], 'tol':[1e-10,1e-1]} # ここを編集する
model = SGDClassifier()
clf = GridSearchCV(model, parameters, cv=3,)
clf.fit(X_train, y_train)
print(clf.best_params_, clf.best_score_)

{'max_iter': 5000, 'penalty': 'l2', 'tol': 0.1} 0.6535542614957877


In [15]:
clf_L2 = SGDClassifier(loss = "log", penalty = "l1", max_iter=30000, fit_intercept=True, random_state=1234, tol=0.1)
clf_L2.fit(X_train,y_train)
y_est_L2 = clf_L2.predict(X_test)
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y_test, y_est_L2)))

正答率 = 66.493%


(ⅱ)交差検証法

In [16]:
scores_L2 = cross_val_score(clf_L2, X_test, y_test)
np.mean(scores_L2)

C:\Users\yuka0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.6626433513580087